In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import *
from sklearn.metrics import *

from catboost import CatBoostClassifier

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test_dataset_test.csv')
sub = pd.read_csv('sample_solution.csv')

print('train:', train_data.shape[0])
print('test:', test_data.shape[0])

train: 955
test: 638


In [3]:
train_data = train_data.drop(['ID', 'ID_y'], axis=1)

In [4]:
# train_data = train_data.fillna(-1)

In [5]:
target_columns = ['Артериальная гипертензия', 'ОНМК',
                  'Стенокардия, ИБС, инфаркт миокарда', 
                  'Сердечная недостаточность',
                  'Прочие заболевания сердца']

In [6]:
dummy_columns = ['Пол', 'Семья', 'Этнос', 'Национальность', 
                 'Религия', 'Образование', 'Профессия',
                 'Статус Курения', 'Частота пасс кур', 'Алкоголь']

In [7]:
time_columns = ['Время засыпания', 'Время пробуждения']

for col in time_columns:
    train_data[col] = pd.to_datetime(train_data[col])
    test_data[col] = pd.to_datetime(test_data[col])

In [8]:
train = pd.concat([train_data.drop(dummy_columns, axis=1), 
                   pd.get_dummies(train_data[dummy_columns])], axis=1)

for col in time_columns:
    train['hour_'+col] = train_data[col].apply(lambda x: x.hour)

In [9]:
test = pd.concat([test_data.drop(dummy_columns, axis=1), 
                   pd.get_dummies(test_data[dummy_columns])], axis=1)

for col in time_columns:
    test['hour_'+col] = test_data[col].apply(lambda x: x.hour)

In [10]:
train = train.drop([x for x in train.drop(target_columns, axis=1).columns if x not in test.columns], axis=1)

In [12]:
cols = train.drop(target_columns+time_columns, axis=1).columns

for i in range(len(cols)-1):
    for j in range(i+1, len(cols)):
        
        train[cols[i]+'_*_'+cols[j]] = train[cols[i]] * train[cols[j]]
        train[cols[i]+'_+_'+cols[j]] = train[cols[i]] + train[cols[j]]
        train[cols[i]+'_-_'+cols[j]] = train[cols[i]] - train[cols[j]]
        train[cols[i]+'_/_'+cols[j]] = train[cols[i]] / train[cols[j]]
        
        test[cols[i]+'_*_'+cols[j]] = test[cols[i]] * test[cols[j]]
        test[cols[i]+'_+_'+cols[j]] = test[cols[i]] + test[cols[j]]
        test[cols[i]+'_-_'+cols[j]] = test[cols[i]] - test[cols[j]]
        test[cols[i]+'_/_'+cols[j]] = test[cols[i]] / test[cols[j]]

/var/folders/8n/c9njytgn0g778_3wm9c937c80000gn/T/ipykernel_56865/694217230.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  test[cols[i]+'_+_'+cols[j]] = test[cols[i]] + test[cols[j]]
/var/folders/8n/c9njytgn0g778_3wm9c937c80000gn/T/ipykernel_56865/694217230.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  test[cols[i]+'_-_'+cols[j]] = test[cols[i]] - test[cols[j]]
/var/folders/8n/c9njytgn0g778_3wm9c937c80000gn/T/ipykernel_56865/694217230.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

In [13]:
train.shape

(955, 10303)

In [20]:
# for i in range(len(target_columns)):

#     target_column = target_columns[i]
    
#     X_train, X_val, y_train, y_val = train_test_split(train.drop(target_columns+time_columns, axis=1), 
#                                                       train[target_column],
#                                                       test_size=0.15, stratify=train[target_column])
    
#     for class_weights in [None, 'Balanced']:
        
#         if not class_weights:
#             print()
#             print(target_column)
    
#         model = CatBoostClassifier(verbose=False,
#                                    early_stopping_rounds=200,
#                                    auto_class_weights=class_weights)
        
#         model.fit(X_train, y_train, eval_set=(X_val, y_val))
        
#         print(class_weights)
#         print(recall_score(y_val, model.predict(X_val)))

In [74]:
fip = pd.read_csv('fip.csv')
good_columns = []
for i in range(fip.shape[1]-1):
    # good_columns.append(list(fip[fip[f'value_{i}'] > 3e-2].feature))
    good_columns.append(list(fip.feature.iloc[list(fip[f'value_{i}'].sort_values(ascending=False).index[:1000])]))

In [75]:
[len(x) for x in good_columns]

[1000, 1000, 1000, 1000, 1000]

In [76]:
models = []
threshs = []

for i in range(len(target_columns)):

    target_column = target_columns[i]
    print(target_column)
    
    models.append([])
    threshs.append([])
    
    mean_recall = 0
    
    for train_idx, val_idx in StratifiedKFold(n_splits=4, shuffle=True, random_state=17).split(train, train[target_column]):
    
        X_train = train.drop(target_columns+time_columns, axis=1).iloc[train_idx][good_columns[i]]
        X_val = train.drop(target_columns+time_columns, axis=1).iloc[val_idx][good_columns[i]]
    
        y_train = train[target_column].iloc[train_idx]
        y_val = train[target_column].iloc[val_idx]
    
        model = CatBoostClassifier(verbose=False,
                                   early_stopping_rounds=200,
                                   auto_class_weights='Balanced'
                                  )
        
        model.fit(X_train, y_train, eval_set=(X_val, y_val))
    
        models[i].append(model)
        
        max_score = 0
        max_th = 0
        for th in range(0, 100, 1):
            val_preds = model.predict_proba(X_val)[:, -1]
            val_preds[val_preds >= th/100] = 1
            val_preds[val_preds < th/100] = 0
            score = recall_score(y_val, val_preds, average='macro')
            if score > max_score:
                max_score = score
                max_th = th/100
        
        threshs[i].append(max_th)
        mean_recall += max_score / 4
    
    print(mean_recall)

Артериальная гипертензия
0.7558213037190031
ОНМК
0.7059016823022224
Стенокардия, ИБС, инфаркт миокарда
0.7221037507562009
Сердечная недостаточность
0.7156994856190683
Прочие заболевания сердца
0.6562709387879556


In [25]:
# Артериальная гипертензия
# 0.749510367897425
# ОНМК
# 0.6779899953337095
# Стенокардия, ИБС, инфаркт миокарда
# 0.6993067504183657
# Сердечная недостаточность
# 0.712200472723321
# Прочие заболевания сердца
# 0.6325360419397117

In [26]:
# for i in range(0, 100, 1):
#     val_preds = model.predict_proba(X_val)[:, -1]
#     val_preds[val_preds >= i/100] = 1
#     val_preds[val_preds < i/100] = 0
#     score = recall_score(y_val, val_preds, average='macro')
#     print(f'th {i}: recall {score}')

In [27]:
# fip = pd.DataFrame()
# fip['feature'] = model.feature_names_
# # fip['value'] = 0
# for i, models_ in enumerate(models):
#     fip[f'value_{i}'] = np.mean([model.feature_importances_ for model in models_], axis=0)
# # fip = fip.sort_values('value', ascending=False)
# fip.to_csv('fip.csv', index=False)

In [79]:
np.mean(threshs[0])

0.48249999999999993

In [80]:
for i in range(len(models)):
    preds_skf = np.zeros(test.shape[0])
    for j in range(len(models[i])):
        model = models[i][j]
        max_th = threshs[i][j]
        # coef = 0.5 / max_th
        preds = model.predict_proba(test[model.feature_names_])[:, -1] 
        # * coef
        # preds[preds >= max_th] = 1
        # preds[preds < max_th] = 0
        preds_skf += preds / 4
    
    preds_skf[preds_skf > np.mean(threshs[i])] = 1
    preds_skf[preds_skf <= np.mean(threshs[i])] = 0
    test[target_columns[i]] = preds_skf

In [81]:
# for i, models_ in enumerate(models):
#     test[target_columns[i]] = np.mean([model.predict_proba(test[model.feature_names_])[:, -1] for model in models_], axis=0)

In [82]:
test[target_columns] = np.round(test[target_columns]).astype(int)

In [83]:
sub = pd.merge(sub['ID'], test[sub.columns],
               on='ID', how='left')

In [84]:
pd.read_csv('sub_5_skf_golden_goodcolumns_th_voting.csv')[target_columns].sum() / sub.shape[0]

Артериальная гипертензия              0.551724
ОНМК                                  0.479624
Стенокардия, ИБС, инфаркт миокарда    0.319749
Сердечная недостаточность             0.297806
Прочие заболевания сердца             0.363636
dtype: float64

In [85]:
sub[target_columns].sum() / sub.shape[0]

Артериальная гипертензия              0.518809
ОНМК                                  0.340125
Стенокардия, ИБС, инфаркт миокарда    0.496865
Сердечная недостаточность             0.299373
Прочие заболевания сердца             0.603448
dtype: float64

In [86]:
train_data[target_columns].sum() / train_data.shape[0]

Артериальная гипертензия              0.467016
ОНМК                                  0.042932
Стенокардия, ИБС, инфаркт миокарда    0.122513
Сердечная недостаточность             0.100524
Прочие заболевания сердца             0.090052
dtype: float64

In [87]:
sub.to_csv('sub_10_4skf_golden_goodcolumns_threshs.csv', index=False)

In [143]:
train_data

,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,0,0,0,Курит,15.0,20.0,0,NaN,употребляю в настоящее время,18.0,2022-07-03 22:00:00,2022-07-03 06:00:00,0,0,0,0,0,0,0,0
1,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,1,0,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,0,NaN,никогда не употреблял,NaN,2022-07-03 00:00:00,2022-07-03 04:00:00,1,0,0,1,1,0,0,0
2,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,1,1-2 раза в неделю,употребляю в настоящее время,17.0,2022-07-03 23:00:00,2022-07-03 07:00:00,0,0,0,0,0,0,0,0
3,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,1,0,0,0,0,1,0,0,Бросил(а),12.0,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,2022-07-03 23:00:00,2022-07-03 07:00:00,0,0,0,1,0,0,0,0
4,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного о...,0,0,1,1,1,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,1,не менее 1 раза в день,употребляю в настоящее время,16.0,2022-07-03 23:00:00,2022-07-03 06:00:00,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,М,в браке в настоящее время,европейская,Русские,Христианство,4 - профессиональное училище,операторы и монтажники установок и машинного о...,1,0,0,0,0,0,0,0,0,0,0,0,1,Курит,16.0,20.0,0,NaN,употребляю в настоящее время,18.0,2022-07-03 22:00:00,2022-07-03 06:00:00,0,0,0,0,0,0,0,0
951,Ж,вдовец / вдова,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,1,0,0,Курит,15.0,10.0,0,NaN,ранее употреблял,26.0,2022-07-03 21:30:00,2022-07-03 07:15:00,0,0,0,0,0,0,0,0
952,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,служащие,0,1,0,0,0,0,0,0,0,0,1,0,0,Бросил(а),20.0,10.0,0,NaN,употребляю в настоящее время,20.0,2022-07-03 22:30:00,2022-07-03 08:00:00,1,0,0,1,0,1,0,0
953,Ж,в браке в настоящее время,европейская,Русские,Атеист / агностик,5 - ВУЗ,"представители законодат. органов власти,...",1,0,0,0,1,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,0,NaN,употребляю в настоящее время,21.0,2022-07-03 23:00:00,2022-07-03 08:30:00,0,1,0,0,0,0,0,0
